# Guide: run anything in distributed fashion

In [1]:
from typing import List

from qiskit import QuantumCircuit, transpile
from qiskit.circuit.random import random_circuit

from quantum_serverless import QuantumServerless, run_qiskit_remote, get, put

In [2]:
serverless = QuantumServerless()

serverless

<QuantumServerless | providers [local]>

In [4]:
circuit = random_circuit(5, 3)

circuit.draw()

┌───┐             ┌───┐              
q_0: ──────────■───────────────┤ S ├─────────────┤ X ├──────────────
               │               └───┘     ┌───────┴───┴────────┐     
q_1: ──────────X─────────────────■───────┤ U2(4.0568,0.20671) ├─────
               │               ┌─┴─┐     └────────────────────┘     
q_2: ──────────X───────────────┤ X ├───────────────X────────────────
            ┌─────┐            └─┬─┘               │           ┌───┐
q_3: ───────┤ Sdg ├──────────────■─────────────────┼───────────┤ T ├
     ┌──────┴─────┴───────┐┌────────────┐          │           └───┘
q_4: ┤ U2(0.37458,3.2557) ├┤ Rz(1.7343) ├──────────X────────────────
     └────────────────────┘└────────────┘

In [7]:
# object

with serverless.context():
    # make any object as remote object
    circuit_reference = put(circuit)

    print("Circuit reference:", circuit_reference)
    
    # to get any object from remote to local call `get` function
    retrieved_object = get(circuit_reference)
    print("Retrieved object\n", retrieved_object)

Circuit reference: ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000002000000)
Retrieved object
                                ┌───┐             ┌───┐              
q_0: ──────────■───────────────┤ S ├─────────────┤ X ├──────────────
               │               └───┘     ┌───────┴───┴────────┐     
q_1: ──────────X─────────────────■───────┤ U2(4.0568,0.20671) ├─────
               │               ┌─┴─┐     └────────────────────┘     
q_2: ──────────X───────────────┤ X ├───────────────X────────────────
            ┌─────┐            └─┬─┘               │           ┌───┐
q_3: ───────┤ Sdg ├──────────────■─────────────────┼───────────┤ T ├
     ┌──────┴─────┴───────┐┌────────────┐          │           └───┘
q_4: ┤ U2(0.37458,3.2557) ├┤ Rz(1.7343) ├──────────X────────────────
     └────────────────────┘└────────────┘                           


In [8]:
circuit_batches = [
    [random_circuit(5, 3) for _ in range(2)]
    for n_batch in range(3)
]
circuit_batches

[[<qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7fdd70e60790>,

In [10]:
# task

# in order to make any function a remote/parallel function annotate it with `remote` decorator

@run_qiskit_remote()
def remote_transpile(circuits: List[QuantumCircuit]):
    return transpile(circuits)

with serverless.context():
    # to call this function by arring `remote` to function call `<function_name>.remote`
    remote_function_reference = remote_transpile(circuit_batches[0])
    print("Function reference:", remote_function_reference)
    
    # to get function results we again using `get`
    result = get(remote_function_reference)
    print("Function result:", result)

Function reference: ObjectRef(c8ef45ccd0112571ffffffffffffffffffffffff0100000001000000)
Function result: [<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fdd70e6db90>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fdd70e60750>]


In [11]:
# we can run multiple remote function in parallel

with serverless.context():
    # to call this function by arring `remote` to function call `<function_name>.remote`
    remote_function_references = [
        remote_transpile(circuit_batch)
        for circuit_batch in circuit_batches
    ]
    print("Functions references:", remote_function_references)
    
    # to get function results we again using `get`
    result = get(remote_function_references)
    print("Functions resulst:", result)

Functions references: [ObjectRef(16310a0f0a45af5cffffffffffffffffffffffff0100000001000000), ObjectRef(c2668a65bda616c1ffffffffffffffffffffffff0100000001000000), ObjectRef(32d950ec0ccf9d2affffffffffffffffffffffff0100000001000000)]
Functions resulst: [[<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fdd70e61b50>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fdd70e77a10>], [<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fdd70e615d0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fdd70dec5d0>], [<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fdd70e761d0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fdd70e61950>]]
